## Part 4 calculating median time to conversion and median conversion price

Conversion = Purchase

1. I am looking at median time per purchase event (accounting for all purchases) and median price per all purchases 
2. In the end I am exporting csv for visualization in Power Bi

In [ ]:
import pandas as pd

In [ ]:
oct = pd.read_csv("2019-Oct.csv", usecols=["event_type", "event_time", "price", "user_id", "user_session"], low_memory=True)

In [79]:

purchases.head()

,event_time,event_type,price,user_id,user_session,start_time,time_to_purchase_min,distance_to_median
0,2019-10-01 00:02:14+00:00,purchase,130.76,543272936,8187d148-3c41-46d4-b0c0-9c08cd9dc564,2019-10-01 00:01:46+00:00,0.466667,2.950000
1,2019-10-01 00:04:37+00:00,purchase,642.69,551377651,3c80f0d6-e9ec-4181-8c5c-837a30be2d68,2019-10-01 00:01:45+00:00,2.866667,0.550000
2,2019-10-01 00:06:02+00:00,purchase,29.51,514591159,0e5dfc4b-2a55-43e6-8c05-97e1f07fbb56,2019-10-01 00:04:02+00:00,2.000000,1.416667
3,2019-10-01 00:07:07+00:00,purchase,54.42,555332717,1dea3ee2-2ded-42e8-8e7a-4e2ad6ae942f,2019-10-01 00:06:00+00:00,1.116667,2.300000
4,2019-10-01 00:09:26+00:00,purchase,189.91,524601178,2af9b570-0942-4dcd-8f25-4d84fba82553,2019-10-01 00:06:31+00:00,2.916667,0.500000


In [33]:
oct.to_parquet("Oct.parquet", index=False)

In [49]:
purchase_sessions = oct.loc[oct['event_type'] == 'purchase', 'user_session'].unique()


In [50]:
df_purchace = oct[oct["user_session"].isin(purchase_sessions)]

In [ ]:
# Get the session start time for each session
session_start = (
    oct.groupby("user_session")["event_time"]
    .min()
    .reset_index(name="start_time")
)

In [53]:
purchase_time = (
    df_purchace[df_purchace['event_type'] == 'purchase']
    .groupby('user_session')['event_time']
    .min()
    .reset_index(name='purchase_time')
)

In [54]:
session_times = pd.merge(
    session_start,
    purchase_time,
    on='user_session',
    how='inner'
)

In [57]:
session_times['purchase_time'] = pd.to_datetime(session_times['purchase_time'])
session_times['start_time'] = pd.to_datetime(session_times['start_time'])

In [59]:
session_times['time_to_purchase'] = (session_times['purchase_time'] - session_times['start_time'])

In [61]:
session_times['time_to_purchase_min'] = (session_times['time_to_purchase'].dt.total_seconds() / 60)

In [62]:
median_time = session_times['time_to_purchase_min'].median()
print(median_time)

2.95


In [63]:
purchases = oct[oct['event_type'] == 'purchase'].copy()

In [64]:
purchases = purchases.merge(
    session_start,
    on='user_session',
    how='left')

In [68]:
purchases["event_time"] = pd.to_datetime(purchases["event_time"])
purchases["start_time"] = pd.to_datetime(purchases["start_time"])

In [ ]:
purchases['time_to_purchase_min'] = ((purchases['event_time'] - purchases['start_time'])
    .dt.total_seconds() / 60)

In [76]:
median_time = purchases['time_to_purchase_min'].median()


In [74]:
purchases['distance_to_median'] = (
    (purchases['time_to_purchase_min'] - median_time).abs()
)

median_purchase = purchases.loc[
    purchases['distance_to_median'].idxmin()
]
print(median_time)

3.4166666666666665


In [73]:
median_price = median_purchase['price']
print(median_price)

1207.71


In [77]:
# Save the DataFrame with time to purchase for each purchase event
purchases.to_csv("purchases_time_to_purchase_oct.csv", index=False)